# Fine-tuning BERT (and friends) for multi-label text classification

In this notebook, we are going to fine-tune BERT to predict one or more labels for a given piece of text. Note that this notebook illustrates how to fine-tune a bert-base-uncased model, but you can also fine-tune a RoBERTa, DeBERTa, DistilBERT, CANINE, ... checkpoint in the same way.

All of those work in the same way: they add a linear layer on top of the base model, which is used to produce a tensor of shape (batch_size, num_labels), indicating the unnormalized scores for a number of labels for every example in the batch.





## Set-up environment

First, we install the libraries which we'll use: HuggingFace Transformers and Datasets.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -U accelerate
!pip install -q transformers datasets
%cd /content/drive/MyDrive/Code/Maitrise/sappelli_email_classification

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.
/content/drive/MyDrive/Code/Maitrise/sappelli_email_classification


## Load dataset

Next, let's download a multi-label text classification dataset from the [hub](https://huggingface.co/).

At the time of writing, I picked a random one as follows:   

* first, go to the "datasets" tab on huggingface.co
* next, select the "multi-label-classification" tag on the left as well as the the "1k<10k" tag (fo find a relatively small dataset).

Note that you can also easily load your local data (i.e. csv files, txt files, Parquet files, JSON, ...) as explained [here](https://huggingface.co/docs/datasets/loading.html#local-and-remote-files).



In [3]:
from datasets import load_dataset

raw_dataset = load_dataset("csv", data_files="./data/annotator_one_hot_encoded.csv")

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
# prompt: Split the dataset, but keep the HuggingFace format
# keep the same splits, but store them in splitted_dataset[train], splitted_dataset[validation], splitted_dataset[test]

from datasets import DatasetDict

dataset_lenght = len(raw_dataset["train"])
print(f"Dataset length: {dataset_lenght}")

train_test_dataset = raw_dataset["train"].train_test_split(test_size=0.2, shuffle=False, seed=42)
val_test_dataset = train_test_dataset["test"].train_test_split(test_size=0.5, shuffle=False, seed=42)


dataset = DatasetDict({
    "train": train_test_dataset["train"],
    "validation": val_test_dataset["train"],
    "test": val_test_dataset["test"]
})

# prompt: length of train, validation, and test

train_length = len(dataset["train"])
val_length = len(dataset["validation"])
test_length = len(dataset["test"])

print(f"Train length: {train_length}")
print(f"Validation length: {val_length}")
print(f"Test length: {test_length}")

Dataset length: 1144
Train length: 915
Validation length: 114
Test length: 115


As we can see, the dataset contains 3 splits: one for training, one for validation and one for testing.

Let's check the first example of the training split:

In [5]:
example = dataset['train'][0]
example

{'body': 'It was cute!\tGenia FitzGerald\t10/10/2000 03:23 PM\t\t \t\t To: Marie Heard/Enron Communications@Enron Communications\t\t cc: Tana Jones/HOU/ECT@ECT\t\t Subject: Re: HOTDOG !This is sooo adorable!Thanks for inviting me to share Mexican food with you.  Tana said it was YOUR fault I wasn t invited!!!!!Genia----- Forwarded by Genia FitzGerald/HOU/ECT on 10/10/2000 03:21 PM -----\t\t----- Forwarded by Genia FitzGerald/HOU/ECT on 10/10/2000 02:30 PM ----- - Hotdog PHOTO2.jpg',
 'Act1A1_amend': False,
 'Act1A1_commit': False,
 'Act1A1_deliver': False,
 'Act1A1_greet': True,
 'Act1A1_propose': False,
 'Act1A1_refuse': False,
 'Act1A1_remind': False,
 'Act1A1_request': False,
 'ReasA1_administrative procedure': False,
 'ReasA1_employment arrangements': False,
 'ReasA1_external collaboration': False,
 'ReasA1_internal collaboration': False,
 'ReasA1_legal procedure': False,
 'ReasA1_logistic arrangements': False,
 'ReasA1_other': False,
 'ReasA1_personal': True,
 'ReasA1_travel plann

The dataset consists of tweets, labeled with one or more emotions.

Let's create a list that contains the labels, as well as 2 dictionaries that map labels to integers and back.

In [6]:
labels = [label for label in dataset['train'].features.keys() if label not in ['body']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['Act1A1_amend',
 'Act1A1_commit',
 'Act1A1_deliver',
 'Act1A1_greet',
 'Act1A1_propose',
 'Act1A1_refuse',
 'Act1A1_remind',
 'Act1A1_request',
 'ReasA1_administrative procedure',
 'ReasA1_employment arrangements',
 'ReasA1_external collaboration',
 'ReasA1_internal collaboration',
 'ReasA1_legal procedure',
 'ReasA1_logistic arrangements',
 'ReasA1_other',
 'ReasA1_personal',
 'ReasA1_travel planning',
 'REA1_accountable non-answer',
 'REA1_ignore',
 'REA1_immediate reply',
 'REA1_postponed reply',
 'NumTA1_0',
 'NumTA1_1',
 'NumTA1_2',
 'NumTA1_3']

## Preprocess data

As models like BERT don't expect text as direct input, but rather `input_ids`, etc., we tokenize the text using the tokenizer. Here I'm using the `AutoTokenizer` API, which will automatically load the appropriate tokenizer based on the checkpoint on the hub.

What's a bit tricky is that we also need to provide labels to the model. For multi-label text classification, this is a matrix of shape (batch_size, num_labels). Also important: this should be a tensor of floats rather than integers, otherwise PyTorch' `BCEWithLogitsLoss` (which the model will use) will complain, as explained [here](https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3).

In [7]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["body"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()

  return encoding

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [8]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/915 [00:00<?, ? examples/s]

Map:   0%|          | 0/114 [00:00<?, ? examples/s]

Map:   0%|          | 0/115 [00:00<?, ? examples/s]

In [9]:
example = encoded_dataset['train'][0]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [10]:
tokenizer.decode(example['input_ids'])

'[CLS] it was cute! genia fitzgerald 10 / 10 / 2000 03 : 23 pm to : marie heard / enron communications @ enron communications cc : tana jones / hou / ect @ ect subject : re : hotdog! this is sooo adorable! thanks for inviting me to share mexican food with you. tana said it was your fault i wasn t invited!!!!! genia - - - - - forwarded by genia fitzgerald / hou / ect on 10 / 10 / 2000 03 : 21 pm - - - - - - - - - - forwarded by genia [SEP]'

In [11]:
example['labels']

[0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0]

In [12]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['Act1A1_greet', 'ReasA1_personal', 'REA1_accountable non-answer', 'NumTA1_0']

Finally, we set the format of our data to PyTorch tensors. This will turn the training, validation and test sets into standard PyTorch [datasets](https://pytorch.org/docs/stable/data.html).

In [13]:
encoded_dataset.set_format("torch")

## Define model

Here we define a model that includes a pre-trained base (i.e. the weights from bert-base-uncased) are loaded, with a random initialized classification head (linear layer) on top. One should fine-tune this head, together with the pre-trained base on a labeled dataset.

This is also printed by the warning.

We set the `problem_type` to be "multi_label_classification", as this will make sure the appropriate loss function is used (namely [`BCEWithLogitsLoss`](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)). We also make sure the output layer has `len(labels)` output neurons, and we set the id2label and label2id mappings.

In [14]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",
                                                           problem_type="multi_label_classification",
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Train the model!

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things:

* `TrainingArguments`, which specify training hyperparameters. All options can be found in the [docs](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments). Below, we for example specify that we want to evaluate after every epoch of training, we would like to save the model every epoch, we set the learning rate, the batch size to use for training/evaluation, how many epochs to train for, and so on.
* a `Trainer` object (docs can be found [here](https://huggingface.co/transformers/main_classes/trainer.html#id1)).

In [15]:
batch_size = 8
metric_name = "f1"

In [16]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    output_dir="./results/annotator_one_hot_encoded",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

We are also going to compute metrics while training. For this, we need to define a `compute_metrics` function, that returns a dictionary with the desired metric values.

In [17]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

Let's verify a batch as well as a forward pass:

In [18]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [19]:
encoded_dataset['train']['input_ids'][0]

tensor([  101,  2009,  2001, 10140,   999,  8991,  2401, 11864,  2184,  1013,
         2184,  1013,  2456,  6021,  1024,  2603,  7610,  2000,  1024,  5032,
         2657,  1013,  4372,  4948,  4806,  1030,  4372,  4948,  4806, 10507,
         1024,  9092,  2050,  3557,  1013,  7570,  2226,  1013, 14925,  2102,
         1030, 14925,  2102,  3395,  1024,  2128,  1024,  2980, 16168,   999,
         2023,  2003, 17111,  2080, 23677,   999,  4283,  2005, 15085,  2033,
         2000,  3745,  4916,  2833,  2007,  2017,  1012,  9092,  2050,  2056,
         2009,  2001,  2115,  6346,  1045,  2347,  1056,  4778,   999,   999,
          999,   999,   999,  8991,  2401,  1011,  1011,  1011,  1011,  1011,
         2830,  2098,  2011,  8991,  2401, 11864,  1013,  7570,  2226,  1013,
        14925,  2102,  2006,  2184,  1013,  2184,  1013,  2456,  6021,  1024,
         2538,  7610,  1011,  1011,  1011,  1011,  1011,  1011,  1011,  1011,
         1011,  1011,  2830,  2098,  2011,  8991,  2401,   102])

In [20]:
#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.7356, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[ 4.4551e-01, -2.5798e-01, -6.4662e-01,  3.2292e-02,  4.7103e-01,
          2.9289e-01,  6.4482e-01, -5.6745e-01, -3.8058e-05, -2.2083e-01,
         -3.6263e-01, -5.3373e-01,  2.3572e-02,  4.8161e-01, -9.4876e-02,
          5.6609e-01,  3.1770e-01, -4.8237e-01,  1.6119e-01,  4.4725e-01,
         -2.4713e-01, -3.7075e-01,  2.0624e-01, -1.6860e-01,  5.2070e-01]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

Let's start training!

In [21]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [22]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.356394,0.189435,0.548217,0.000000
2,No log,0.339523,0.130518,0.530435,0.000000
3,No log,0.332753,0.117409,0.529506,0.000000
4,No log,0.327803,0.277397,0.578974,0.000000
5,0.374800,0.325544,0.286689,0.582494,0.000000


TrainOutput(global_step=575, training_loss=0.36938897506050444, metrics={'train_runtime': 179.0438, 'train_samples_per_second': 25.552, 'train_steps_per_second': 3.212, 'total_flos': 300995414611200.0, 'train_loss': 0.36938897506050444, 'epoch': 5.0})

In [27]:
# prompt: save model

model.save_pretrained("./models/annotator_one")

## Evaluate

After training, we evaluate our model on the validation set.

In [23]:
trainer.evaluate()

{'eval_loss': 0.3255438506603241,
 'eval_f1': 0.28668941979522183,
 'eval_roc_auc': 0.5824941220675955,
 'eval_accuracy': 0.0,
 'eval_runtime': 0.8933,
 'eval_samples_per_second': 127.615,
 'eval_steps_per_second': 16.791,
 'epoch': 5.0}

## Inference

Let's test the model on a new sentence:

In [24]:
text = "Hey Henri, I am writing to you because I am interested in the position of a Data Scientist at your company. I have a strong background in machine learning and I am proficient in Python. I am looking forward to hearing from you. Best, John Doe."

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the `batch_size` equals 1. The logits is a tensor that contains the (unnormalized) scores for every individual label.

In [25]:
logits = outputs.logits
logits.shape

torch.Size([1, 25])

To turn them into actual predicted labels, we first apply a sigmoid function independently to every score, such that every score is turned into a number between 0 and 1, that can be interpreted as a "probability" for how certain the model is that a given class belongs to the input text.

Next, we use a threshold (typically, 0.5) to turn every probability into either a 1 (which means, we predict the label for the given example) or a 0 (which means, we don't predict the label for the given example).

In [26]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

['NumTA1_1']
